# Building Feature Based Grammars

## Grammatical Features

Starting off with a very simple example, using dictionaries to store features and their values.

kim = {'CAT': 'NP', 'ORTH': 'Kim', 'REF': 'k'}
chase = {'CAT': 'V', 'ORTH': 'chased', 'REL': 'chase'}

The objects kim and chase both have a couple of shared features, CAT (grammatical category) and ORTH (orthography, i.e., spelling). In addition, each has a more semantically-oriented feature: kim['REF'] is intended to give the referent of kim, while chase['REL'] gives the relation expressed by chase.

We might want to add further information to the above. For example, in the case of a verb, it is often useful to know what "semantic role" is played by the arguments of the verb. In the case of chase, the subject plays the role of "agent", while the object has the role of "patient".

In [3]:
chase['AGT'] = 'sbj'
chase['PAT'] = 'obj'

If we now process a sentence *Kim chased Lee*, we want to bind the verb's agent role, to the subject and the patient role to the object. 

We do this by linking the `REF` feature to the relevant NP.

In [4]:
lee = {'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'}

In [6]:
sent = "Kim chased Lee"
tokens = sent.split()

In [9]:
def lex2fs(word):
    for fs in [kim, lee, chase]:
        if fs['ORTH'] == word:
            return fs

In [10]:
subj, verb, obj = lex2fs(tokens[0]), lex2fs(tokens[1]), lex2fs(tokens[2])
verb['AGT'] = subj['REF']
verb['PAT'] = obj['REF']

In [11]:
for k in ['ORTH', 'REL', 'AGT', 'PAT']:
     print("%-5s => %s" % (k, verb[k]))

ORTH  => chased
REL   => chase
AGT   => k
PAT   => l
